# DS 2002 Data Project 1

## 	Create a Data Warehouse Using Data from Various Sources

The goal of this project is to demonstrate (1) an understanding of and (2) competence creating and implementing basic data science systems such as pipelines, scripts, data transformations, APIs, databases and cloud services.

### Prerequisites:
#### Import the Necessary Libraries

In [1]:
import os
import json
import numpy
import datetime
import pandas as pd

import pymongo
from sqlalchemy import create_engine

#### Declare & Assign Connection Variables for the MongoDB Server, the MySQL Server & Databases

In [2]:
host_name = "localhost"
ports = {"mongo" : 27017, "mysql" : 3306}

user_id = "root"
pwd = "password!"

src_dbname = "chinook"
dst_dbname = "chinook_dw2"

#### Create the New Data Warehouse database

Database to work with: create **chinook_dw2** database and use it as the target of all subsequent operations

In [3]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

sqlEngine.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
sqlEngine.execute(f"CREATE DATABASE `{dst_dbname}`;")
sqlEngine.execute(f"USE {dst_dbname};")

#### Define Functions for Getting Data From and Setting Data Into Databases

In [4]:
def get_sql_dataframe(user_id, pwd, host_name, db_name, sql_query):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    conn = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, conn);
    conn.close()
    
    return dframe


def get_mongo_dataframe(user_id, pwd, host_name, port, db_name, collection, query):
    '''Create a connection to MongoDB, with or without authentication credentials'''
    if user_id and pwd:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db_name)
        client = pymongo.MongoClient(mongo_uri)
    else:
        conn_str = f"mongodb://{host_name}:{port}/"
        client = pymongo.MongoClient(conn_str)
    
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    client.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

### Populate MongoDB with Source Data

Be certain you run this cell **ONLY ONCE!** Otherwise, you will fill your MongoDB database with duplicate records which will cause duplicate key errors when you attempt to create and populate the MySQL data warehouse dimension and fact tables.

From the original Chinook database, the following cell pushes employee and customer information to MongoDB

In [ ]:
port = ports["mongo"]
conn_str = f"mongodb://{host_name}:{port}/"
client = pymongo.MongoClient(conn_str)
db = client[src_dbname]

data_dir = os.path.join(os.getcwd(), 'chinook')

json_files = {"customers" : 'customers.json',
              "employees" : 'employees.json'
             }

for file in json_files:
    json_file = os.path.join(data_dir, json_files[file])
    with open(json_file, 'r') as openfile:
        json_object = json.load(openfile)
        file = db[file]
        result = file.insert_many(json_object)
        #print(f"{file} was successfully loaded.")

        
client.close() 

### 1.0 Use MongoDB to create dimension tables: dim_customers and dim_employees

#### 1.1 Extract Data

In [5]:
# define variables to use get_mongo_dataframe function
query = {}
port = ports["mongo"]

In [6]:
# customer data
df_customers = get_mongo_dataframe(None, None, host_name, port, src_dbname, "customers", query)
df_customers.head(2)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,,Theodor-Heuss-Straße 34,Stuttgart,,Germany,70174,+49 0711 2842222,,leonekohler@surfeu.de,5


In [7]:
# employee data
df_employees = get_mongo_dataframe(None, None, host_name, port, src_dbname, "employees", query)
df_employees.head(2)

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,0,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com


#### 1.2 Transformations

In [8]:
# customers
df_customers.rename(columns={"CustomerId":"customer_key","FirstName":"customer_FirstName","LastName":"customer_LastName"}, 
                    inplace=True)
df_customers.drop('Fax', axis=1, inplace=True)
df_customers.head(2)

,customer_key,customer_FirstName,customer_LastName,Company,Address,City,State,Country,PostalCode,Phone,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,luisg@embraer.com.br,3
1,2,Leonie,Köhler,,Theodor-Heuss-Straße 34,Stuttgart,,Germany,70174,+49 0711 2842222,leonekohler@surfeu.de,5


In [9]:
# employees
df_employees.rename(columns={"EmployeeId":"employee_key","FirstName":"employee_FirstName","LastName":"employee_LastName",
                             "ReportsTo":"ReportsTo_employee_key"}, inplace=True)

drop_cols = ['BirthDate','HireDate']
df_employees.drop(drop_cols, axis=1, inplace=True)
df_employees.head(2)

,employee_key,employee_LastName,employee_FirstName,Title,ReportsTo_employee_key,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,0,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com


#### 1.3 Load transformed DataFrames into the New Data Warehouse by Creating New Tables

In [10]:
dataframe = df_customers
table_name = 'dim_customers'
primary_key = 'customer_key'
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

In [11]:
dataframe = df_employees
table_name = 'dim_employees'
primary_key = 'employee_key'
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

#### 1.4 Validate New Dimension Tables were Created 

In [12]:
sql_customers = "SELECT * FROM chinook_dw2.dim_customers;"
df_dim_customers = get_sql_dataframe(user_id, pwd, host_name, dst_dbname, sql_customers)
df_dim_customers.head(2)

,customer_key,customer_FirstName,customer_LastName,Company,Address,City,State,Country,PostalCode,Phone,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,luisg@embraer.com.br,3
1,2,Leonie,Köhler,,Theodor-Heuss-Straße 34,Stuttgart,,Germany,70174,+49 0711 2842222,leonekohler@surfeu.de,5


In [13]:
sql_employees = "SELECT * FROM chinook_dw2.dim_employees;"
df_dim_employees = get_sql_dataframe(user_id, pwd, host_name, dst_dbname, sql_employees)
df_dim_employees.head(2)

,employee_key,employee_LastName,employee_FirstName,Title,ReportsTo_employee_key,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,0,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com


### 2.0 Create & populate dim_tracks (another dimension table) and fact_invoices (fact_table)

#### 2.1 dim_tracks

Query the source **chinook** database to fill a dataframe for each of the source tables needed to create the **dim_tracks** dimension table: track, album, artist, genre, and mediatype. These dataframes are joined using the merge( ) method of the Pandas DataFrame. The dataframe is then pushed back to the MySQL server to create and populate the new dimension table.

##### 2.1.1 get all data from each of the tables

In [14]:
sql_tracks = "SELECT * FROM chinook.track;"
df_tracks = get_sql_dataframe(user_id, pwd, host_name, src_dbname, sql_tracks)
df_tracks.rename(columns = {"TrackId":"track_key","Name":"track_name","AlbumId":"album_key", "MediaTypeId":"mediatype_key","GenreId":"genre_key"}, inplace = True)
df_tracks.head(2)

,track_key,track_name,album_key,mediatype_key,genre_key,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99


In [16]:
sql_albums = "SELECT * FROM chinook.album;"
df_albums = get_sql_dataframe(user_id, pwd, host_name, src_dbname, sql_albums)
df_albums.rename(columns = {"AlbumId":"album_key","Title":"album_title","ArtistId":"artist_key"}, inplace = True)
df_albums.head(2)

,album_key,album_title,artist_key
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2


In [17]:
sql_artists = "SELECT * FROM chinook.artist;"
df_artists = get_sql_dataframe(user_id, pwd, host_name, src_dbname, sql_artists)
df_artists.rename(columns = {"ArtistId":"artist_key","Name":"artist_name"}, inplace = True)
df_artists.head(2)

,artist_key,artist_name
0,1,AC/DC
1,2,Accept


In [18]:
sql_genre = "SELECT * FROM chinook.genre;"
df_genre = get_sql_dataframe(user_id, pwd, host_name, src_dbname, sql_genre)
df_genre.rename(columns = {"GenreId":"genre_key","Name":"genre"}, inplace = True)
df_genre.head(2)

,genre_key,genre
0,1,Rock
1,2,Jazz


In [19]:
sql_mediatype = "SELECT * FROM chinook.mediatype;"
df_mediatype = get_sql_dataframe(user_id, pwd, host_name, src_dbname, sql_mediatype)
df_mediatype.rename(columns = {"MediaTypeId":"mediatype_key","Name":"mediatype"}, inplace = True)
df_mediatype.head(2)

,mediatype_key,mediatype
0,1,MPEG audio file
1,2,Protected AAC audio file


##### 2.1.2 get album title and artist id

In [20]:
df_tracks = pd.merge(df_tracks, df_albums, on='album_key', how='left')
df_tracks.head(2)

,track_key,track_name,album_key,mediatype_key,genre_key,Composer,Milliseconds,Bytes,UnitPrice,album_title,artist_key
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99,Balls to the Wall,2


##### 2.1.3 get artist name

In [21]:
df_tracks = pd.merge(df_tracks, df_artists, on='artist_key', how='left')
df_tracks.head(2)

,track_key,track_name,album_key,mediatype_key,genre_key,Composer,Milliseconds,Bytes,UnitPrice,album_title,artist_key,artist_name
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,For Those About To Rock We Salute You,1,AC/DC
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99,Balls to the Wall,2,Accept


##### 2.1.4 get media type

In [22]:
df_tracks = pd.merge(df_tracks, df_mediatype, on='mediatype_key', how='left')
df_tracks.head(2)

,track_key,track_name,album_key,mediatype_key,genre_key,Composer,Milliseconds,Bytes,UnitPrice,album_title,artist_key,artist_name,mediatype
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,For Those About To Rock We Salute You,1,AC/DC,MPEG audio file
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99,Balls to the Wall,2,Accept,Protected AAC audio file


##### 2.1.5 get genre

In [23]:
df_tracks = pd.merge(df_tracks, df_genre, on='genre_key', how='left')
df_tracks.head(2)

,track_key,track_name,album_key,mediatype_key,genre_key,Composer,Milliseconds,Bytes,UnitPrice,album_title,artist_key,artist_name,mediatype,genre
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,For Those About To Rock We Salute You,1,AC/DC,MPEG audio file,Rock
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99,Balls to the Wall,2,Accept,Protected AAC audio file,Rock


In [24]:
# drop and reorder columns
drop_columns = ['album_key','mediatype_key','genre_key','artist_key']
df_tracks.drop(drop_columns, axis=1, inplace=True)

# reordering
df_tracks = df_tracks[['track_key', 'track_name', 'album_title', 'artist_name', 'Composer','genre','Milliseconds','Bytes',
                      'UnitPrice','mediatype']]

In [25]:
df_tracks.head(5)

,track_key,track_name,album_title,artist_name,Composer,genre,Milliseconds,Bytes,UnitPrice,mediatype
0,1,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",Rock,343719,11170334,0.99,MPEG audio file
1,2,Balls to the Wall,Balls to the Wall,Accept,None,Rock,342562,5510424,0.99,Protected AAC audio file
2,3,Fast As a Shark,Restless and Wild,Accept,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",Rock,230619,3990994,0.99,Protected AAC audio file
3,4,Restless and Wild,Restless and Wild,Accept,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",Rock,252051,4331779,0.99,Protected AAC audio file
4,5,Princess of the Dawn,Restless and Wild,Accept,Deaffy & R.A. Smith-Diesel,Rock,375418,6290521,0.99,Protected AAC audio file


##### 2.1.6 write dataframe back to MySQL database

In [26]:
tablename = "dim_tracks"
pk_column = "track_key"
db_operation = "insert"
set_dataframe(user_id, pwd, host_name, dst_dbname, df_tracks, tablename, pk_column, db_operation)

In [27]:
# verify this executed:
sql_dim_tracks = "SELECT * FROM chinook_dw2.dim_tracks;"
df_dim_tracks = get_sql_dataframe(user_id, pwd, host_name, dst_dbname, sql_dim_tracks)
df_dim_tracks.head(2)

,track_key,track_name,album_title,artist_name,Composer,genre,Milliseconds,Bytes,UnitPrice,mediatype
0,1,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",Rock,343719,11170334,0.99,MPEG audio file
1,2,Balls to the Wall,Balls to the Wall,Accept,None,Rock,342562,5510424,0.99,Protected AAC audio file


#### 2.2 fact_invoices

The same method used to create the **dim_tracks** table will now be used to create the **fact_invoices** table. This requires two source tables from the **chinook** database: invoice and invoiceline.

##### 2.2.1 get data from each table

In [28]:
sql_invoices = "SELECT * FROM chinook.invoice;"
df_invoices = get_sql_dataframe(user_id, pwd, host_name, src_dbname, sql_invoices)
df_invoices.rename(columns = {"InvoiceId":"invoice_key","CustomerId":"customer_key"}, inplace = True)
df_invoices.head(2)

,invoice_key,customer_key,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96


In [29]:
sql_invoicelines = "SELECT * FROM chinook.invoiceline;"
df_invoicelines = get_sql_dataframe(user_id, pwd, host_name, src_dbname, sql_invoicelines)
df_invoicelines.rename(columns = {"InvoiceLineId":"fact_invoice_key","InvoiceId":"invoice_key",
                                  "TrackId":"track_key"}, inplace = True)
df_invoicelines.head(2)

,fact_invoice_key,invoice_key,track_key,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1


##### 2.2.2 merge tables

In [30]:
df_invoices = pd.merge(df_invoices, df_invoicelines, on='invoice_key', how='right')
df_invoices.head(2)

,invoice_key,customer_key,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total,fact_invoice_key,track_key,UnitPrice,Quantity
0,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,1,2,0.99,1
1,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,2,4,0.99,1


In [31]:
# reordering columns:
df_invoices = df_invoices[['fact_invoice_key','invoice_key','customer_key', 'track_key', 'InvoiceDate', 'BillingAddress',
'BillingCity','BillingState','BillingCountry','BillingPostalCode','UnitPrice','Quantity', 'Total']]
df_invoices.head(5)

,fact_invoice_key,invoice_key,customer_key,track_key,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,UnitPrice,Quantity,Total
0,1,1,2,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,0.99,1,1.98
1,2,1,2,4,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,0.99,1,1.98
2,3,2,4,6,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,0.99,1,3.96
3,4,2,4,8,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,0.99,1,3.96
4,5,2,4,10,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,0.99,1,3.96


##### 2.2.3 Write dataframe back to database

In [32]:
tablename = "fact_invoices"
pk_column = "fact_invoice_key"
db_operation = "insert"
set_dataframe(user_id, pwd, host_name, dst_dbname, df_invoices, tablename, pk_column, db_operation)

In [33]:
# verify this executed:
sql_invoices = "SELECT * FROM chinook_dw2.fact_invoices;"
df_fact_invoices = get_sql_dataframe(user_id, pwd, host_name, dst_dbname, sql_invoices)
df_fact_invoices.head(2)

,fact_invoice_key,invoice_key,customer_key,track_key,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,UnitPrice,Quantity,Total
0,1,1,2,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,0.99,1,1.98
1,2,1,2,4,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,0.99,1,1.98


### 3.0 Load dim_date (date dimension table) from local json

The last table needed in the data warehouse is the date dimension table. The table was created in SQL using code from: http://arcanecode.com/2009/11/18/populating-a-kimball-date-dimension/
and exported to a csv. 

#### 3.1 load dim_date table from local file system

In [34]:
file_path = "C:/Users/Student/Documents/chinook/dim_date.json"
df_dim_date = pd.read_json(file_path)
df_dim_date.head()

,date_key,full_date,date_name,date_name_us,date_name_eu,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,...,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
0,20000101,2000-01-01,2000/01/01,01/01/2000,01/01/2000,7,Saturday,1,1,Weekend,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
1,20000102,2000-01-02,2000/01/02,01/02/2000,02/01/2000,1,Sunday,2,2,Weekend,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
2,20000103,2000-01-03,2000/01/03,01/03/2000,03/01/2000,2,Monday,3,3,Weekday,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
3,20000104,2000-01-04,2000/01/04,01/04/2000,04/01/2000,3,Tuesday,4,4,Weekday,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
4,20000105,2000-01-05,2000/01/05,01/05/2000,05/01/2000,4,Wednesday,5,5,Weekday,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3


#### 3.2 Integrate date dimension in fact_invoices

Now that we have the date dimension table, we need to replace the **InvoiceDate** column in **fact_invoices** with the **date_key** from **dim_date**.

##### 3.2.1 merge tables

In [37]:
df_invoices.rename(columns={"InvoiceDate":"full_date"}, inplace=True)
df_dim_date['full_date'] = df_dim_date['full_date'].astype('datetime64[ns]')
df_invoices = pd.merge(df_invoices, df_dim_date, on='full_date', how='inner')
df_invoices.head(2)

,fact_invoice_key,invoice_key,customer_key,track_key,full_date,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,...,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
0,1,1,2,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,...,N,1,2009,2009-01,2009Q1,7,3,2009,2009-07,2009Q3
1,2,1,2,4,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,...,N,1,2009,2009-01,2009Q1,7,3,2009,2009-07,2009Q3


In [38]:
# drop all added columns except for date key
drop_columns = ['date_name',
 'date_name_us',
 'date_name_eu',
 'day_of_week',
 'day_name_of_week',
 'day_of_month',
 'day_of_year',
 'weekday_weekend',
 'week_of_year',
 'month_name',
 'month_of_year',
 'is_last_day_of_month',
 'calendar_quarter',
 'calendar_year',
 'calendar_year_month',
 'calendar_year_qtr',
 'fiscal_month_of_year',
 'fiscal_quarter',
 'fiscal_year',
 'fiscal_year_month',
 'fiscal_year_qtr']
df_invoices.drop(drop_columns, axis=1, inplace=True)

In [39]:
df_invoices = df_invoices[['fact_invoice_key','invoice_key','customer_key', 'track_key', 'full_date','date_key',
                           'BillingAddress', 'BillingCity','BillingState','BillingCountry','BillingPostalCode','UnitPrice','Quantity', 'Total']]
df_invoices.head(2)

,fact_invoice_key,invoice_key,customer_key,track_key,full_date,date_key,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,UnitPrice,Quantity,Total
0,1,1,2,2,2009-01-01,20090101,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,0.99,1,1.98
1,2,1,2,4,2009-01-01,20090101,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,0.99,1,1.98


In [40]:
# the date_key is correct, so we can drop the full_date column:
df_invoices.drop('full_date', axis=1, inplace=True)
df_invoices.head(2)

,fact_invoice_key,invoice_key,customer_key,track_key,date_key,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,UnitPrice,Quantity,Total
0,1,1,2,2,20090101,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,0.99,1,1.98
1,2,1,2,4,20090101,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,0.99,1,1.98


#### 3.3 Write dataframes back to database

Now we push the dim_date dimension table and updated fact_invoices table with the date dimension to MySQL.

In [41]:
# dim_date
tablename = "dim_date"
pk_column = "date_key"
db_operation = "insert"
set_dataframe(user_id, pwd, host_name, dst_dbname, df_dim_date, tablename, pk_column, db_operation)

In [42]:
# verify execution:
sql_dates = "SELECT * FROM chinook_dw2.dim_date;"
df_dates = get_sql_dataframe(user_id, pwd, host_name, dst_dbname, sql_dates)
df_dates.head(2)

,date_key,full_date,date_name,date_name_us,date_name_eu,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,...,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
0,20000101,2000-01-01,2000/01/01,01/01/2000,01/01/2000,7,Saturday,1,1,Weekend,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
1,20000102,2000-01-02,2000/01/02,01/02/2000,02/01/2000,1,Sunday,2,2,Weekend,...,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3


In [43]:
# updated fact_invoices
## DROP TABLE FIRST:
sqlEngine.execute(f"DROP TABLE IF EXISTS `{dst_dbname}.fact_invoices`;")

In [44]:
# insert table
tablename = "fact_invoices"
pk_column = "fact_invoice_key"
db_operation = "insert"
set_dataframe(user_id, pwd, host_name, dst_dbname, df_invoices, tablename, pk_column, db_operation)

In [45]:
# verify execution
sql_invoices = "SELECT * FROM chinook_dw2.fact_invoices;"
df_fact_invoices = get_sql_dataframe(user_id, pwd, host_name, dst_dbname, sql_invoices)
df_fact_invoices.head(2)

,fact_invoice_key,invoice_key,customer_key,track_key,date_key,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,UnitPrice,Quantity,Total
0,1,1,2,2,20090101,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,0.99,1,1.98
1,2,1,2,4,20090101,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,0.99,1,1.98


### 4.0. Demonstrate that the New Data Warehouse Exists and Contains the Correct Data

To demonstrate the data warehouse is functioning properly, this SQL query will return:

- Each Customer’s Last Name
- The total amount of tracks each customer has purchased
- The total amount of money each customer has spent
- The name of the support representative (employee) responsible for the customer 

In [46]:
validate_data = """
   
   SELECT 
        c.customer_LastName,
        SUM(i.Quantity) AS total_quantity,
        ROUND(SUM(i.UnitPrice),2) AS total_spent,
        e.employee_LastName
    FROM
        chinook_dw2.fact_invoices AS i
    LEFT OUTER JOIN chinook_dw2.dim_customers AS c
        ON i.customer_key = c.customer_key
	LEFT OUTER JOIN chinook_dw2.dim_employees AS e
		ON c.SupportRepId = e.employee_key
    GROUP BY c.customer_LastName
    ORDER BY total_spent DESC;

    """

fact_invoice_summary = get_sql_dataframe(user_id, pwd, host_name, dst_dbname, validate_data)
display(fact_invoice_summary)

,customer_LastName,total_quantity,total_spent,employee_LastName
0,Holý,38.0,49.62,Johnson
1,Cunningham,38.0,47.62,Park
2,Rojas,38.0,46.62,Johnson
3,O'Reilly,38.0,45.62,Peacock
4,Kovács,38.0,45.62,Peacock
5,Zimmermann,38.0,43.62,Peacock
6,Barnett,38.0,43.62,Johnson
7,Ralston,38.0,43.62,Peacock
8,Stevens,38.0,42.62,Johnson
9,Gruber,38.0,42.62,Johnson
